In [ ]:
pip install bs4 requests fake_useragent

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=9d395977adef40c3d19ee804e3206d0b9f87ead181aedd19b1a7158c3e60c470
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import re
import time
import random

In [ ]:
def ProductName(soup) :
  p_name = ''
  try :
    p_name = soup.find('span', id = 'productTitle').get_text().strip()
  except :
    p_name = soup.find('span', class_ = 'a-size-large product-title-word-break').get_text().strip()

  return p_name

In [ ]:
def ProductPrice(soup) :
  p_price = ''
  try :
    p_price = soup.find('span', class_ = 'a-offscreen').get_text()
  except :
    p_price = soup.find('span', class_="a-price-whole").get_text()

  return p_price

In [ ]:
def ProductRating(soup):
  p_rating = ''
  try :
    p_rating = soup.find('span', class_= "a-size-medium a-color-base").get_text()
  except :
    p_rating = soup.find('span', class_ = "a-icon-alt").get_text()

  return p_rating

In [ ]:
def ProductRatingCount(soup) :
  num_of_ratings = ''
  try :
    num_of_ratings = soup.find('span', class_ = 'a-size-base').get_text()
  except :
    num_of_ratings = soup.find('span', class_ = "a-size-base a-color-secondary").get_text()

  return num_of_ratings

In [ ]:
def Description(soup) :
  desc = ''
  try :
    disc_div = product_soup.find('div', id = 'feature-bullets')
    description = disc_div.find_all('span', 'a-list-item')
    if description : desc = description[1].get_text()
  except :
    desc = soup.find('span', class_ = "a-size-small social-proofing-faceout-title-text")
    if desc :
      desc = desc.get_text()
  return desc

In [ ]:
def ProductManufacturer(soup) :
  p_manufacturer = ''
  try :
    div = soup.find('div', id = "detailBullets_feature_div")
    list_items = div.find_all('li')
    for item in list_items :
      text = item.get_text().strip()
      if text.startswith() == 'Manufacturer' :
        manufacturer = text.split(':')[1].strip()
        p_manufacturer = manufacturer.replace('\u200e', '').strip()
      if len(p_manufacturer):
        break

  except :
    div = soup.find('div', class_ = 'class="a-expander-content a-expander-section-content a-section-expander-inner"')
    table_rows = div.find_all('tr')
    for row in table_rows :
      head = row.find('th').get_text().strip()
      if head == 'Manufacturer' :
        data = row.find('td').get_text().strip()
        p_manufacturer = data.replace('&lrm;', '').strip()
        break

  return p_manufacturer

In [ ]:
def ProductAsin(link) :
  web_address = link.split('/')
  asin = ''
  for i in web_address[-1] :
    if i == '?' :
      break
    asin += i
  return asin

In [ ]:
def ProductDescription(soup) :
  p_description = ''
  try :
    p_description = soup.find('div', class_="a-section a-spacing-small").find('span').get_text()
  except :
    desc_div = soup.find('div', class_ = "a-section a-spacing-medium a-spacing-top-small")
    desc_list = disc_div.find_all('span', 'a-list-item')
    p_description = desc[-1].get_text()

  return p_description

In [ ]:
product_table = {'Product_URL' : [],
'Product_Name' : [],
'Product_Price' : [],
'Product_Rating' : [],
'Product_Ratings_Count' : [],
'Description' : [],
'ASIN' : [],
'Product_Description' : [],
'Manufacturer' : [] }

In [ ]:
user_agent_list = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15']

In [ ]:
i = 1
while i < 21 :
  URL = "https://www.amazon.in/s?k=bags&page=" + str(i)
  HEADERS = ({'UserAgent' : random.choice(user_agent_list), 'Accept-Language': 'en-US, en;q=0.5'})
  page = requests.get(URL, headers = HEADERS)
  if page.status_code != 200 :
    time.sleep(3)
    i -= 1
    continue
  soup = BeautifulSoup(page.content, "lxml")
  links = soup.find_all('a', class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
  product_links = []
  for link in links :
    product_links.append("https://www.amazon.in/"+link.get('href'))
  for product in product_links :
    HEADERS = ({'UserAgent' : random.choice(user_agent_list), 'Accept-Language': 'en-US, en;q=0.5'})

    product_page = requests.get(product, headers = HEADERS)
    if product_page.status_code != 200 :
      time.sleep(3)
      continue
    product_soup = BeautifulSoup(product_page.content, "lxml")

    product_table['Product_URL'].append(product)
    product_table['Product_Name'].append(ProductName(product_soup))
    product_table['Product_Price'].append(ProductPrice(product_soup))
    product_table['Product_Rating'].append(ProductRating(product_soup))
    product_table['Product_Ratings_Count'].append(ProductRatingCount(product_soup))
    product_table['Description'].append(Description(product_soup))
    product_table['ASIN'].append(ProductAsin(product))
    product_table['Product_Description'].append(ProductDescription(product_soup))
    product_table['Manufacturer'].append(ProductManufacturer(product_soup))
    time.sleep(3)
    i += 1
    #   Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.75 Safari/537.36
    #   Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0
    #   Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/114.0
    #   Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36




In [ ]:
amazon_products_firstpage = pd.DataFrame.from_dict(product_table)
amazon_products_firstpage